# Python Environments in VS Code

https://code.visualstudio.com/docs/python/environments 

# Python Libraries 

python -m pip install --upgrade pip \
python -m pip install influxdb-client[ciso] \
python -m pip install pandas[performance] \
python -m pip install --upgrade matplotlib \
python -m pip install bokeh \
python -m pip install seaborn \
python -m pip install scipy
